<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 6


## NLP and Machine Learning on [travel.statsexchange.com](http://travel.stackexchange.com/) data

---

In Project 7 you'll be doing NLP and machine learning on post data from stackexchange's travel subdomain. 

This project is setup like a mini Kaggle competition. You are given the training data and when projects are submitted your model will be tested on the held-out testing data. There will be prizes for the people who build models that perform best on the held out test set!

---

## Notes on the data

The data is again compressed into the `.7z` file format to save space. There are 6 .csv files and one readme file that contains some information on the fields.

    posts_train.csv
    comments_train.csv
    users.csv
    badges.csv
    votes_train.csv
    tags.csv
    readme.txt
    
The data is located in your datasets folder:

    DSI-SF-2/datasets/stack_exchange_travel.7z
    
If you're interested in where this data came from and where to get more data from other stackexchange subdomains, see here:

https://ia800500.us.archive.org/22/items/stackexchange/readme.txt


### Recommended Utilities for .7z

- For OSX [Keka](http://www.kekaosx.com/en/) or [The Unarchiver](http://wakaba.c3.cx/s/apps/unarchiver.html). 
- For Windows [7-zip](http://www.7-zip.org/) is the standard. 
- For Linux try the `p7zip` utility.  `sudo apt-get install p7zip`.



<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 1. Use LDA to find what topics are discussed on travel.stackexchange.com.

---

Text can be found in the posts and the comments datasets. The `ParentId` column in the posts dataset indicates what the "question" post was for a given post. Comment text can be merged onto the post they are part of with the `PostId` field.

The text may have some HTML tags. BeautifulSoup has convenient ways to get rid of markup or extract text if you need to. You can also parse the strings yourself if you like.

The tags dataset has the "tags" that the users have officially given the post.

**1.1 Implement LDA against the text features of the dataset(s).**

- This can be posts or a combination of posts and comments if you want more power.
- Find optimal **K/num_topics**.

**1.2 Compare your topics to the tags. Do the LDA topics make sense? How do they compare to the tags?**


In [9]:
import pandas as pd
import numpy as np

In [10]:
posts = pd.read_csv("stack_exchange_travel/posts_train.csv")
comments = pd.read_csv("stack_exchange_travel/comments_train.csv")
tags = pd.read_csv("stack_exchange_travel/tags.csv")
votes = pd.read_csv("stack_exchange_travel/votes_train.csv")

In [3]:
comments.head(1)

,CreationDate,Id,PostId,Score,Text,UserDisplayName,UserId
0,2011-06-21T20:25:14.257,1,1,0,To help with the cruise line question: Where a...,NaN,12.0


In [11]:
posts_array = posts[["Id", "Body"]]
comments_array = comments[["PostId", "Text"]]        

In [12]:
allcomments = pd.DataFrame(comments_array.groupby("PostId")["Text"].sum())

In [13]:
allcomments["PostId"] = allcomments.index
allcomments.head(1)

,Text,PostId
PostId,,
1,To help with the cruise line question: Where a...,1


In [14]:
posts_array.dropna(axis = 0, inplace = True)

/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
posts_comments = pd.merge(posts_array, allcomments, how = "left", left_on = "Id", right_on = "PostId")

In [16]:
posts_comments.head(1)

,Id,Body,Text,PostId
0,1,<p>My fiancée and I are looking for a good Car...,To help with the cruise line question: Where a...,1.0


In [17]:
text_arrays = zip(posts_comments["PostId"].values,
    posts_comments["Body"].values,
    posts_comments["Text"].values)
                

fulltext = []

for p_id, post, comment in text_arrays:
    if np.isnan(p_id):
        fulltext.append(post)
    else:
        fulltext.append(post + comment)

In [18]:
from bs4 import BeautifulSoup

def removehtml(x):
    soup = BeautifulSoup(x, "lxml")
    return soup.get_text()

cleantext = map(removehtml, fulltext)

In [19]:
data = {"id": list(posts_array["Id"].values),
       "text": cleantext}

finalframe = pd.DataFrame(data)

In [20]:
#time to vectorize
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words = "english", ngram_range=(1,2), 
                      max_df = 0.8, min_df = 0.03)

vectorizer = vect.fit(finalframe["text"].values)
vectorized = vect.transform(finalframe["text"].values)

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 2. What makes an answer likely to be "accepted"?

---

**2.1 Build a model to predict whether a post will be marked as the answer.**

- This is a classification problem.
- You're free to use any of the machine learning algorithms or techniques we have learned in class to build the best model you can.
- NLP will be very useful here for pulling out useful and relevant features from the data. 
- Though not required, using bagging and boosting models like Random Forests and Gradient Boosted Trees will _probably_ get you the highest performance on the test data (but who knows!).


**2.2 Evaluate the performance of your classifier with a confusion matrix and accuracy. Explain how your model is performing.**

**2.3 Plot either a ROC curve or precision-recall curve (or both!) and explain what they tell you about your model.**

NOTE: You should only be predicting this for `PostTypeID=2` posts, which are the "answer" posts. This doesn't mean, however, that you can't or shouldn't use the parent questions as predictors!


In [21]:
accepted = list(posts["AcceptedAnswerId"].unique())
accepted = [i for i in accepted if not np.isnan(i)]
posts["accepted"] = np.zeros(posts.shape[0], dtype = np.int8)
posts.loc[posts["Id"].isin(accepted), "accepted"] = 1

In [95]:
posts.columns

Index([u'AcceptedAnswerId', u'AnswerCount', u'Body', u'ClosedDate',
       u'CommentCount', u'CommunityOwnedDate', u'CreationDate',
       u'FavoriteCount', u'Id', u'LastActivityDate', u'LastEditDate',
       u'LastEditorDisplayName', u'LastEditorUserId', u'OwnerDisplayName',
       u'OwnerUserId', u'ParentId', u'PostTypeId', u'Score', u'Tags', u'Title',
       u'ViewCount', u'accepted'],
      dtype='object')

In [22]:
#remove posts with missing bodies
posts = posts.loc[~ posts.Body.isnull(), :]

y = posts.loc[posts.PostTypeId == 2, "accepted"]
X = posts.loc[posts.PostTypeId == 2, ["Score", "CommentCount"]]

corpus = posts.loc[posts.PostTypeId == 2, "Body"].values

vectorized_answers = vect.transform(corpus)

print X.values.shape
print vectorized_answers.shape
print y.shape

X = np.hstack((X.values, vectorized_answers.todense()))

(23967, 2)
(23967, 449)
(23967,)


In [32]:
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score



In [25]:
#time to fit some models

#logistic regression baseline:
logit = LogisticRegression()
print "baseline accuracy: ", np.mean(cross_val_score(logit, X, y, cv = 3))


baseline accuracy:  0.735761672299


In [33]:
#gradient boosted tree

param_grid = {
    "n_estimators": [50, 100],
    "learning_rate": [0.5, 0.1],
    "max_depth": [4, 8, 10],
    "subsample": [0.5, 0.75, 1],
    "reg_lambda": [0.1, 1, 10]
}

xgb = XGBClassifier()
grd = GridSearchCV(xgb, param_grid = param_grid, verbose=2, n_jobs = -1, cv = 2)

print "xgboost accuracy: ", np.mean(cross_val_score(xgb, X, y, cv = 3))

xgboost accuracy:  0.734092710811


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 3. What is the score of a post?

---

**3.1 Build a model that predicts the score of a post.**

- This is a regression problem now. 
- You can and should be predicting score for both "question" and "answer" posts, so keep them both in your dataset.
- Again, use any techniques that you think will get you the best model.

**3.2 Evaluate the performance of your model with cross-validation and report the results.**

**3.3 What is important for determining the score of a post, if anything?**


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 4. How many views does a post have?

---

**4.1 Build a model that predicts the number of views a post has.**

- This is another regression problem. 
- Predict the views for all posts, not just the "answer" posts.

**4.2 Evaluate the performance of your model with cross-validation and report the results.**

**4.3 What is important for the number of views a post has, if anything?**

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5. Build a pipeline or other code to automate evaluation of your models on the test data.

---

Now that you've constructed your three predictive models, build a pipeline or code that can easily load up the raw testing data and evaluate your models on it.

The testing data that is held out is in the same raw format as the training data you have. _Any cleaning and preprocessing that you did on the training data will need to be done on the testing data as well!_

This is a good opportunity to practice building pipelines, but you're not required to. Custom functions and classes are fine as long as they are able to process and test the new data.


<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Lets Model - Tournament for stock market predictions

>Start this section of the project by downloading the train and test datasets from the following site: https://numer.ai/rules

> - The data set is clean, your goal is to develop a classification model(s) 
> - Report all the results including log loss, and other coefficients you consider iteresting